In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
import torch.nn as nn
import pyepo

from src.data.config import HP
from sklearn.model_selection import train_test_split
from src.models.LinearRegression import LinearRegression

from src.solvers.spnia_asym import AsymmetricSPNI
from src.models.ShortestPathGrb import shortestPathGrb
from src.models.ShortestPathGrid import ShortestPathGrid
from src.solvers.BendersDecomposition import BendersDecomposition
from data.DataGenerator import DataGenerator
from models.SPOTrainer import SPOTrainer
from models.POTrainer import POTrainer

from scripts.compare_po_spo import compare_po_spo

In [ ]:
# Define hyperparameters
c_min: float = 1.0
c_max: float = 10.0
d_min: float = 1.0
d_max: float = 10.0
Q = 0.6
B = 5
network = (6, 8)
random_seed = 31

# ML hyperparameters
num_features = 5
num_data_samples = 100
test_size = 0.2
data_loader_batch_size = 32
epochs = 5

# Set the random seed for reproducibility
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)


In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
graph = ShortestPathGrid(network[0], network[1])
opt_model = shortestPathGrb(graph)

In [ ]:
# # Generate artificial data samples
# data_gen = DataGenerator(
#     num_costs=graph.num_cost,
#     num_features=num_features,
#     cost_feature_map="PolynomialKernel",
#     c_range=(c_min, c_max),
#     epsilon_bar=0.05
# )
# costs, features = data_gen.generate_data(num_samples=num_data_samples)

features, costs = pyepo.data.shortestpath.genData(
    1000, 
    num_features, 
    (graph.m, graph.n), 
    deg=3, 
    noise_width=0.05, 
    seed=31
)

# Split the data into training and testing sets
X_train, X_test, c_train, c_test = train_test_split(features, costs, test_size=test_size, random_state=random_seed)

# Create data loaders for training and testing
optnet_train_dataset = pyepo.data.dataset.optDataset(opt_model, X_train, c_train)
optnet_test_dataset = pyepo.data.dataset.optDataset(opt_model, X_test, c_test)

g = torch.Generator().manual_seed(random_seed)
optnet_train_loader = torch.utils.data.DataLoader(optnet_train_dataset, batch_size=data_loader_batch_size, shuffle=True, generator=g)
optnet_test_loader = torch.utils.data.DataLoader(optnet_test_dataset, batch_size=data_loader_batch_size, shuffle=False, generator=g)


## Predict-then-Optimize

In [ ]:
# Define your network dimensions
input_size  =  num_features   # e.g. number of features in your cost‐vector
hidden_size =  64   # number of neurons in the hidden layer
output_size =  graph.num_cost   # e.g. # of target outputs, or number of classes

# Build the model with nn.Sequential
po_model = nn.Sequential(
    nn.Linear(input_size, hidden_size),  # first affine layer
    nn.ReLU(),                           # non‐linearity
    nn.Linear(hidden_size, output_size),  # second affine layer
    nn.Sigmoid()                         # output activation function
)

# Define the loss function and optimizer
po_criterion = nn.MSELoss()
optimizer = torch.optim.Adam(po_model.parameters(), lr=0.1)

In [ ]:
po_trainer = POTrainer(
    pred_model=po_model,
    opt_model=opt_model,
    optimizer=optimizer,
    loss_fn=po_criterion
)

train_loss_log, train_regret_log, test_loss_log, test_regret_log = po_trainer.fit(optnet_train_loader, optnet_test_loader, epochs=epochs)

# Plot the learning curve
POTrainer.vis_learning_curve(
    po_trainer,
    train_loss_log,
    train_regret_log,
    test_loss_log,
    test_regret_log
)

print("Final regret on test set: ", test_regret_log[-1])

## Smart Predict-then-Optimize with OptNet

In [ ]:
# Define your network dimensions
input_size  =  num_features   # e.g. number of features in your cost‐vector
hidden_size =  64   # number of neurons in the hidden layer
output_size =  graph.num_cost   # e.g. # of target outputs, or number of classes

# Build the model with nn.Sequential
spo_model = nn.Sequential(
    nn.Linear(input_size, hidden_size),  # first affine layer
    nn.ReLU(),                           # non‐linearity
    nn.Linear(hidden_size, output_size),  # second affine layer
    nn.Sigmoid()                         # output activation function
)

# Init SPO+ loss
spop = pyepo.func.SPOPlus(opt_model, processes=1)

# Init optimizer
optimizer = torch.optim.Adam(spo_model.parameters(), lr=.1)

In [ ]:
# Create a trainer instance
spo_trainer = SPOTrainer(pred_model=spo_model, 
                  opt_model=opt_model, 
                  optimizer=optimizer, 
                  loss_fn=spop
                )

train_loss_log, train_regret_log, test_loss_log, test_regret_log = spo_trainer.fit(optnet_train_loader, optnet_test_loader, epochs=epochs)

# Plot the learning curve
SPOTrainer.vis_learning_curve(
    spo_trainer,
    train_loss_log,
    train_regret_log,
    test_loss_log,
    test_regret_log
)

print("Final regret on test set: ", test_regret_log[-1])

## Compare both in application

In [ ]:
# Set the configuration parameters
test_data_samples = 10 # number of training data
deg=3
noise_width=0.05
seed=31

In [ ]:
# Initialize the configuration class
cfg = HP()

# # Change configuration parameters (optional)
# cfg.set("test_data_samples", test_data_samples)
# cfg.set("deg", deg)
# cfg.set("noise_width", noise_width)
# cfg.set("seed", seed)
# cfg.set("num_features", num_features)
# cfg.set("grid_size", (graph.m, graph.n))
# cfg.set("random_seed", random_seed)

# cfg.set("num_data_samples", num_data_samples)
# cfg.set("test_size", test_size)
# cfg.set("data_loader_batch_size", data_loader_batch_size)

In [ ]:
true_objs, po_objs, spo_objs = compare_po_spo(cfg, opt_model, po_model, spo_model)

In [ ]:
po_val = [(po - true) / true for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true for spo, true in zip(spo_objs, true_objs)]

plt.scatter(np.ones_like(po_objs), po_val)
plt.scatter(np.ones_like(spo_objs) + 1, spo_val)
# plt.legend(['PO', 'SPO'], location='north')

In [ ]:
sjkhg

In [ ]:
# Set parameters for data generation
test_data_samples = 10 # number of training data
cost_index = 1 # index of the cost vector to use

# Generate data for shortest path problem
features, costs = pyepo.data.shortestpath.genData(
    test_data_samples, 
    num_features, 
    (graph.m, graph.n), 
    deg=3, 
    noise_width=0.05, 
    seed=31
)

# Choose a cost vector
true_costs = costs[cost_index]
features = features[cost_index]

# Set the cost for the grid (Optionally specify the source and target nodes)
opt_model.setObj(true_costs)

# Solve shortest path problem
path, obj = opt_model.solve(versatile=True, title="Shortest path on graph with chosen weights")


In [ ]:
# Predict shortest path with predict-then-optimize framework
predicted_costs = po_model(torch.tensor(features, dtype=torch.float32)).detach().numpy()
po_graph = ShortestPathGrid(network[0], network[1], cost=predicted_costs)
po_shortest_path = shortestPathGrb(po_graph)
po_path, po_obj = po_shortest_path.solve()

In [ ]:
# Predict shortest path with smart predict-then-optimize framework
predicted_costs = spo_model(torch.tensor(features, dtype=torch.float32)).detach().numpy()
spo_graph = ShortestPathGrid(network[0], network[1], cost=predicted_costs)
spo_shortest_path = shortestPathGrb(spo_graph)
spo_path, spo_obj = spo_shortest_path.solve()

In [ ]:
# Print results
print(f"Objective (Predict-then-Optimize shortest path): \t{opt_model.evaluate(po_path)}")
print(f"Objective (Smart Predict-then-Optimize shortest path): \t{opt_model.evaluate(spo_path)}")

# Visualize results
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
po_shortest_path.visualize(ax=ax[0], 
                     colored_edges=po_path,
                     title="Predict-then-Optimize shortest path")
spo_shortest_path.visualize(ax=ax[1], 
                   colored_edges=spo_path,
                   title="Smart Predict-then-Optimize shortest path")
fig.tight_layout()

In [ ]:
# TODO: Add random seeds everywhere (also pytorch) to ensure reproducibility (Use Codex for this?)
# TODO: Replace single-instance example above with a statistical argument
# Idea: What about training a PO framework first and then tuning the parameters with SPO? 
#       -> Gives more accurate & easier to interpret results and can finetune for SPO afterwards.
#       -> What about computing both gradients and changing their weights for learning gradually over time, 
#               e.g., 100% PO first, 50/50 in the middle, and 100% SPO at the end?